In [2]:
import pandas as pd

working_dataframe = pd.read_csv('/Users/johannessolibieda/Documents/GitHub/Its-Wekk/Fucking Final Data/Working_DataFrame.csv')
target_dataframe = pd.read_csv('/Users/johannessolibieda/Documents/GitHub/Its-Wekk/Fucking Final Data/CleanedFinal_Target_Data_Combined_resid_Trend')

In [3]:
from sklearn.feature_selection import SelectKBest, f_regression

# Select only numeric columns from working_dataframe
working_dataframe_numeric = working_dataframe.select_dtypes(include=[float, int])

# Add 'Datum' column back if it exists in the original dataframe
if 'Datum' in working_dataframe.columns:
    working_dataframe_numeric['Datum'] = working_dataframe['Datum']

# Define the feature columns and target column
feature_columns = working_dataframe_numeric.columns.tolist()
if 'Datum' in feature_columns:
    feature_columns.remove('Datum')
target_column = 'PM10_Combined_Trend_Residual'

# Align the indices of working_dataframe_numeric and target_dataframe
aligned_data = working_dataframe_numeric.join(target_dataframe[target_column], how='inner')

# Prepare the data
X = aligned_data[feature_columns]
y = aligned_data[target_column]

# Initialize SelectKBest with the scoring function and the desired number of features
selector = SelectKBest(score_func=f_regression, k=50)

# Fit the selector
selector.fit(X, y)

# Get the selected features
selected_features = [feature for feature, support in zip(feature_columns, selector.get_support()) if support]

# Create a new dataframe with the selected features
if 'Datum' in working_dataframe.columns:
    reduced_working_dataframe = working_dataframe_numeric[selected_features + ['Datum']]
else:
    reduced_working_dataframe = working_dataframe_numeric[selected_features]

print("Selected features:", selected_features)
print("Reduced dataframe shape:", reduced_working_dataframe.shape)
print("Selected features as list:", selected_features)
for feature in selected_features:
    print(feature)


Selected features: ['Basel Temperature [2 m elevation corrected]', 'Basel Precipitation Total', 'Basel Wind Speed [10 m]', 'Basel Wind Direction [10 m]', 'Stromverbrauch', '350n_sumLief', '350v_sumLW', '352v_sumPW', '352v_sumLief', '354n_sumLW', '402v_sumPW', '402n_sumLief', '402n_sumLW', '403n_sumLW', '403v_sumPW', '403v_sumLW', '405v_sumLief', '405v_sumLW', '406n_sumLW', '406v_sumLief', '406v_sumLW', '408n_sumPW', '408n_sumLief', '408n_sumLW', '409v_sumLW', '415v_sumLief', '415v_sumLW', '415n_sumLW', '417n_sumPW', '417n_sumLief', '417n_sumLW', '419v_sumLW', '419n_sumPW', '419n_sumLief', '420n_sumLief', '420n_sumLW', '420v_sumLW', '653n_sumLW', '659v_sumLW', '659n_sumLief', '659n_sumLW', '660v_sumPW', '660v_sumLW', '660n_sumPW', '660n_sumLW', '84111104n_sumLief', '84111104v_sumLief', '84111108v_sumLief', 'Gasverbrauch', 'Traffic']
Reduced dataframe shape: (24574, 51)
Selected features as list: ['Basel Temperature [2 m elevation corrected]', 'Basel Precipitation Total', 'Basel Wind Spe

In [4]:
# Reorder columns to make 'Datum' the first column
columns = ['Datum'] + [col for col in reduced_working_dataframe.columns if col != 'Datum']
reduced_working_dataframe = reduced_working_dataframe[columns]

# Export the reduced working dataframe with the most relevant features to a CSV file
reduced_working_dataframe.to_csv('50MostImpFeatures_DF.csv', index=False)

In [5]:
# Align the target dataframe with the reduced working dataframe
aligned_target_dataframe = target_dataframe.loc[reduced_working_dataframe.index]

# Export the aligned target dataframe to a CSV file
aligned_target_dataframe.to_csv('TargetCutto50MostImpFeatures_DF.csv', index=False)